### Setup a Conda Environment on Coiled

In [ ]:
import coiled

coiled.create_software_environment(
   name='mapshader-tiling',
   conda={
       'channels': ['conda-forge', 'defaults'],
       'dependencies': [
           'python=3.9',
           'mapshader',
           'dask=2022.04.2',
           'distributed=2022.4.2',
           'cloudpickle=2.0.0',
           'spatialpandas',
           'boto3',
       ],
    },
)

### Create Dask Cluster on Coiled 

In [ ]:
from coiled.v2 import Cluster
cluster = Cluster(name='mapshader-tiler',
                  n_workers=10,
                  worker_cpu=2,
                  worker_options={"nthreads": 1},
                  scheduler_memory="8 GiB",
                  software='mapshader-tiling')

from dask.distributed import Client
client = Client(cluster)
print('Dashboard:', client.dashboard_link)

### Clear cluster memory if necessary

In [ ]:
client.restart()

## Tile World Cities (Sparse Points)

### Setup Mapshader Source

In [ ]:
import geopandas as gpd
import mapshader
import spatialpandas

from mapshader.sources import VectorSource


def world_cities_source():

    # construct transforms
    reproject_transform = dict(name='reproject_vector', args=dict(epsg=3857))
    add_xy_fields_transform = dict(name='add_xy_fields', args=dict(geometry_field='geometry'))
    buffered_extent_transform = dict(name='add_projected_buffered_extent', 
                                     args=dict(crs='4326',
                                               buffer_distance=.01,
                                               geometry_field='geometry'))
    sp_transform = dict(name='to_spatialpandas', args=dict(geometry_field='geometry'))
    
    transforms = [reproject_transform,
                  add_xy_fields_transform,
                  buffered_extent_transform,
                  sp_transform]

    # construct value obj
    source_obj = dict()
    source_obj['name'] = 'World Cities'
    source_obj['key'] = 'world-cities'
    source_obj['text'] = 'World Cities'
    source_obj['description'] = 'World Cities Point Locations'
    source_obj['geometry_type'] = 'point'
    source_obj['agg_func'] = 'max'
    source_obj['cmap'] = ['aqua', 'aqua']
    source_obj['shade_how'] = 'linear'
    source_obj['dynspread'] = 2
    source_obj['raster_interpolate'] = 'linear'
    source_obj['xfield'] = 'X'
    source_obj['yfield'] = 'Y'
    source_obj['filepath'] = gpd.datasets.get_path('naturalearth_cities')
    source_obj['transforms'] = transforms
    source_obj['service_types'] = ['tile', 'wms', 'image', 'geojson']
    source_obj['tiling'] = dict(min_zoom=1,
                                max_zoom=2,
                                xmin_field='buffer_0_4326_xmin',
                                xmax_field='buffer_0_4326_xmax',
                                ymin_field='buffer_0_4326_ymin',
                                ymax_field='buffer_0_4326_ymax'
                               )

    return source_obj


cities_source = VectorSource.from_obj(world_cities_source())
cities_source.load()
cities_source.data

### Create a Dask DataFrame of Tile to Process based on Map Source feature extents

In [ ]:
from mapshader.tile_utils import list_tiles

tiles_ddf = list_tiles(cities_source, npartitions=2)
tiles_ddf.persist()

### Generate tile images and save to an S3 bucket

In [ ]:
from mapshader.tile_utils import save_tiles_to_outpath

save_tiles_to_outpath(cities_source, tiles_ddf, outpath='s3://mapshader-tiling-test-999/cities/')

### View tiles on OSM basemap

In [ ]:
from ipyleaflet import Map, TileLayer, basemaps, basemap_to_tiles

tiles_url = 'https://mapshader-tiling-test-999.s3.amazonaws.com/cities/{z}/{x}/{y}.png'
tile_layer=TileLayer(url=tiles_url)

from ipyleaflet import Map, basemaps, basemap_to_tiles

m = Map(
    basemap=basemap_to_tiles(basemaps.OpenStreetMap.Mapnik),
    center=(48.204793, 350.121558),
    zoom=3
    )
m

m = Map(
    basemap=basemap_to_tiles(basemaps.OpenStreetMap.Mapnik),
    zoom=2,
    scroll_wheel_zoom=True)

m.add_layer(tile_layer)
m
display(m)